In [8]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torchvision import transforms

In [2]:
import os

# go to root directory if needed
print(f"Current working directory: {os.getcwd()}")
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir("..")
    print(f"Changed working directory to: {os.getcwd()}")

Current working directory: /Users/patrickschuermann/Documents/GitHub/FlareSense/notebooks
Changed working directory to: /Users/patrickschuermann/Documents/GitHub/FlareSense


In [3]:
import src.utils.data as data


In [10]:
# Definieren Sie das PyTorch Lightning-Modell
class ResNet50BinaryClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # Laden Sie das vordefinierte ResNet-50-Modell ohne den letzten Klassifikations-Layer
        self.resnet50 = models.resnet50(pretrained=True)
        num_features = self.resnet50.fc.in_features
        self.resnet50.fc = nn.Linear(num_features, 1)  # 1 Ausgabe für binäre Klassifikation

    def forward(self, x):
        return self.resnet50(x)

    def training_step(self, batch, batch_idx):
        images, _, labels, _ = batch
        binary_labels = (labels != 0)
        outputs = self(images)
        loss = nn.BCEWithLogitsLoss()(outputs, binary_labels)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        return optimizer

# Erstellen Sie eine Instanz des Modells
model = ResNet50BinaryClassifier()

data_folder_path = "data/raw/burst_images/"

data_module = data.ECallistoDataModule(
    data_folder=data_folder_path,
    transform=transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((193, 240), antialias=True),
            transforms.ToTensor(),
        ]
    ),
    batch_size=32,
    num_workers=0,
    val_ratio=0.15,
    test_ratio=0.15,
)
data_module.setup()

# Erstellen Sie einen Trainer für das Training
trainer = pl.Trainer(max_epochs=2)

# Laden Sie den DataLoader für die Trainingsdaten
train_loader = data_module.train_dataloader()

# Starten Sie das Training
trainer.fit(model, train_loader)


/Users/patrickschuermann/.local/share/virtualenvs/Deep_Learning-EdiEON_k/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/patrickschuermann/.local/share/virtualenvs/Deep_Learning-EdiEON_k/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type   | Params
------------------------------------
0 | resnet50 | ResNet | 23.5 M
----

Epoch 0:   0%|          | 0/1892 [00:00<?, ?it/s] 

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 32, 193, 240] to have 3 channels, but got 32 channels instead